# [<small>weed</small>](https://tonyfast.github.io/wip/weed)

A collection public data sources and visualization about nugs.

https://gist.githubusercontent.com/rogerallen/1583593/raw/74f2ef57ac82d60dc1e7ae871d1a9c60c9953fac/us_state_abbrev.py

In [1]:
from wip.weed import gallup, usa, history

In [2]:
from us_state_abbrev import us_state_abbrev as abbr
history.ts = history.ts[history.ts.index.astype(int) > gallup.df.index.year.min()]
history.laws_by_state.columns = history.laws_by_state.columns.map(abbr.get)
history.laws_by_state = history.laws_by_state[history.laws_by_state.index > gallup.df.index.min()]

In [3]:
from bokeh.plotting import *
output_notebook(resources=__import__('bokeh').resources.CDN)

Loading BokehJS ...

In [4]:
source = gallup.df.pipe(ColumnDataSource)

laws = history.laws_by_state.apply(lambda s: s[s.gt(0)].index.tolist(), axis=1).apply(','.join).to_frame('text').pipe(ColumnDataSource)

p = figure(x_axis_type='datetime', output_backend='webgl', width=800, height=400)
p.yaxis[0].axis_label, p.title.text = '%', gallup.__question__.replace('\\', '')
p.line('time', '-', source=source)
p.text('time', '-', dict(value='👎'), source=source)

p.line('time', '+', source=source)
p.text('time', '+', dict(value='👍'), source=source)

p.text('year', 100, source=laws, text_font_size='5px')
show(p)